In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

In [ ]:
import nltk
import string
import re
import csv
import spacy
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, average_precision_score
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import  confusion_matrix
np.random.seed(500)
nltk.download('stopwords')
nltk.download('punkt')
stoplist = stopwords.words('english')
nlp = spacy.load('en_core_web_sm')
ps =PorterStemmer()

column_names = ["Abb", "sense" ,"ABB_frm" ,"start_pos","end_pos", "info","context"]
txt_path='/content/drive/My Drive/Colab Notebooks/AnonymizedClinicalAbbreviationsAndAcronymsDataSet.txt'
data_pd=pd.read_csv( txt_path, sep="|", header=None,encoding='cp1252',)

print(data_pd.shape)

In [ ]:
sub_pd=data_pd.loc[data_pd[0]=='SS']
#sub_pd.head()

In [ ]:
# detect annotation error

for index,row in sub_pd.iterrows():
    doc=nlp(str(row[6]))
    for k,s in enumerate(doc.sents):
        
        for j,token in enumerate(s):
            
            if(token.idx==row[3] and token.text != row[0]):
                
                print(index, token.idx,token.text,s[j+1],s[j+1].idx)
                
                # modify 
                sub_pd.at[index,3]=s[j+1].idx
                

In [ ]:
i=[34024,34185,34496,34499]
for j in i:
  sub_pd=sub_pd.drop(j)


In [ ]:
#sub_18_all=pd.DataFrame(sub_pd)
sub_18_all=sub_18_all.append(sub_pd)
sub_18_all.shape


In [ ]:
sub_18_all.to_csv(r'sub_18_all.txt', header=None, index=None, sep='|', mode='a')

In [ ]:
embedding_matrix = np.zeros((1, 50))
for index, row in sub_pd.iterrows():
  get_sum_embedding(row[6],row[3])
  embedding_matrix = np.vstack([embedding_matrix, get_sum_embedding(row[6],row[3])])

In [ ]:
embedding_matrix.shape
x = np.delete(embedding_matrix, (0), axis=0)


In [ ]:
sense= []
for index, row in sub_pd.iterrows():
  sense.append(str(row[1]))
len(sense)

In [ ]:
X_train, X_test, Train_Y, Test_Y= train_test_data(x,sense)

In [ ]:
def train_test_data(feature_list,sense):
  v = DictVectorizer(sparse=None)
  X=feature_list
  #X = v.fit_transform(feature_list)
  Encoder = LabelEncoder()
  X_train, X_test, y_train, y_test = train_test_split(X, sense, test_size = 0.2, random_state=0)
  Train_Y = Encoder.fit_transform(y_train)
  Test_Y = Encoder.fit_transform(y_test)
  #print(len(X_train ),len(y_train))
  return X_train, X_test, Train_Y, Test_Y

In [ ]:
print(SVM_class(X_train, X_test, Train_Y, Test_Y))
print(G_NB(X_train,Train_Y,X_test,Test_Y))
print(KN_class(X_train,Train_Y,X_test,Test_Y))

In [ ]:
def SVM_class(X_train, X_test, Train_Y, Test_Y):
  SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
  SVM.fit(X_train,Train_Y)
  # predict the labels on validation dataset
  predictions_SVM = SVM.predict(X_test)
  # Use accuracy_score function to get the accuracy
  #print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
  #print(average_precision_score(predictions_SVM, Test_Y))
  #print (classification_report(Test_Y, predictions_SVM))
  return str(accuracy_score(predictions_SVM, Test_Y)*100)

def G_NB(X_train,Train_Y,X_test,Test_Y):
  #Create a Gaussian Classifier
   model = GaussianNB()

   # Train the model using the training sets
   model.fit(X_train,Train_Y)

   #Predict Output
   predicted= model.predict(X_test) 
   #print("Accuracy:",metrics.accuracy_score(Test_Y, predicted))
   return str(metrics.accuracy_score(Test_Y, predicted))

def KN_class(X_train,Train_Y,X_test,Test_Y):
  classifier = KNeighborsClassifier(n_neighbors=5)
  classifier.fit(X_train,Train_Y)
  y_pred = classifier.predict(X_test)
  #print("Accuracy:",metrics.accuracy_score(Test_Y, y_pred))
  return str(metrics.accuracy_score(Test_Y, y_pred))


In [ ]:
def get_sum_embedding(sent,pos):
  doc=nlp(sent)
  list_str=[]
  vv = np.zeros((1,50))
  for k,s in enumerate(doc.sents):
    for j,token in enumerate(s):
      if (token.idx==pos):
        print(s,len(s),token.idx, token.text)
        for i in range(0,len(s)):
          if str(s[i])==token.text or check_special(str(s[i]))==False:
            continue
          else:
            #get embedding
            vv=vv+get_embedding('/content/drive/My Drive/Colab Notebooks/glove.6B.50d.txt',str(s[i]).lower(), 50)
            #print(s[i],get_embedding('/content/drive/My Drive/Colab Notebooks/glove.6B.50d.txt',str(s[i]).lower(), 50))
  return vv

In [ ]:
#Get Embedding Vector
def get_embedding(filepath, word_, embedding_dim):
  #vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
  v = np.zeros((1,50))
  with open(filepath) as f:
    for line in f:
      word, *vector = line.split()
      if word ==word_:
         v = np.array(vector, dtype=np.float32)[:embedding_dim]

  return v

In [ ]:
#ignore special sympols
def check_special(wordSpecial):
  t= False
  if(bool(re.search('^[a-zA-Z0-9]*$',wordSpecial))==True):
    t=True
  return t

In [ ]:
#customize sentence boundaries
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
print("Before:", [sent.text for sent in doc.sents])

def set_custom_boundaries(doc):
    for token in doc[:-1]:
        if token.text=='%' or token.text.endswith('%'):
            doc[token.i+1].is_sent_start = False
    return doc

nlp.add_pipe(set_custom_boundaries, before="parser")
doc = nlp(text)
print("After:", [sent.text for sent in doc.sents])

In [ ]:
#under developing to check sentence boundaries
for token in doc:
 print(token.is_sent_start, ' '+token.text)

In [ ]:
sense= []
embedding_matrix = np.zeros((1, 50))

for index,row in sub_pd.iterrows():
  L_Side,R_Side=get_sides(str(row[6]))
  embedding_matrix = np.vstack([embedding_matrix, get_embedding_feature(L_Side,R_Side)])

x = np.delete(embedding_matrix, (0), axis=0)

for index, row in sub_pd.iterrows():
  sense.append(str(row[1]))


In [ ]:
#prepare data for the model and train the model

X_train, X_test, Train_Y, Test_Y= train_test_data(x,sense)

print(SVM_class(X_train, X_test, Train_Y, Test_Y))
print(G_NB(X_train,Train_Y,X_test,Test_Y))
print(KN_class(X_train,Train_Y,X_test,Test_Y))

In [ ]:
#get left and right side
def get_sides(sent):
  doc=nlp(sent)
  for k,s in enumerate(doc.sents):
    for j,token in enumerate(s):
      if (token.idx==132):
        if j !=0:
          L_Side=s[0:j-1]
        else:
          L_Side=[]
        if j != len(s):
          R_Side=s[j+1:len(s)-1]
        else:
          R_Side=[]
  return L_Side,R_Side

In [ ]:
#ignore special sympols
def check_special(wordSpecial):
  t= False
  if (bool(re.search('^[a-zA-Z0-9]*$',wordSpecial))==True) and wordSpecial.isdigit()==False:
      t=True
  return t

In [ ]:
#get_feature
def get_embedding_feature(l,r):
  l,r=get_sides(text)
  window_size=r_window_size=0
  for i in range(len(l) ,0 , -1):
    if check_special(str(l[i])) and window_size < 5:
      l_em=get_embedding('/content/drive/My Drive/Colab Notebooks/glove.6B.50d.txt', str(l[i]).lower(), 50)
      window_size=window_size+1

  for j in range(0,len(r) ):
    if check_special(str(r[j])) and r_window_size < 5: 
      r_em=get_embedding('/content/drive/My Drive/Colab Notebooks/glove.6B.50d.txt', str(r[j]).lower(), 50)
      r_window_size = r_window_size+1

  total_em=l_em+r_em/(window_size+r_window_size)
  return total_em




CORRECT  some annotations

In [ ]:
#customize sentence boundaries
import spacy
nlp = spacy.load("en_core_web_sm")
#doc = nlp(text)
#print("Before:", [sent.text for sent in doc.sents])

def set_custom_boundaries(doc):
    for token in doc[:-1]:
        if token.text=='%' or token.text.endswith('%'):
            doc[token.i+1].is_sent_start = False
    return doc

nlp.add_pipe(set_custom_boundaries, before="parser")

In [ ]:
import pandas as pd
txt_path='/content/drive/My Drive/Colab Notebooks/AnonymizedClinicalAbbreviationsAndAcronymsDataSet.txt'
data_pd=pd.read_csv( txt_path, sep="|", header=None,encoding='cp1252',)
# detect annotation error



In [ ]:
#BAL', 'BK', 'C3', 'CEA', 'CVA', 'CVP', 'CVS', 'DIP', 'ER', 'FISH','NAD', 'OTC', 'PE', 'PT', 'SBP', 'SS', 'ASA', 'AMA'
#sub_pd=data_pd.loc[data_pd[0]=='ASA']
sub_pd.head()

for index,row in sub_pd.iterrows():
    doc=nlp(str(row[6]))
    for k,s in enumerate(doc.sents):
        
        for j,token in enumerate(s):
            
            if(token.idx==row[3] and token.text != row[0]):
                
                print(index, token.idx,token.text,s[j+1],s[j+1].idx)
                
                # modify 
                sub_pd.at[index,3]=s[j+1].idx
                

In [ ]:
#sub_18_abb=pd.DataFrame()
sub_18_abb=sub_18_abb.append(sub_pd)
print(sub_18_abb.shape)
print(sub_18_abb[0].unique())

In [ ]:
sub_pd=sub_pd.drop(index=2916)

In [ ]:
for index,row in sub_pd.iterrows():
    doc=nlp(str(row[6]))
    for k,s in enumerate(doc.sents):
        
        for j,token in enumerate(s):
            
            if(token.idx==row[3] and token.text != row[0]):
                
                print(index, token.idx,token.text,s[j+1],s[j+1].idx)
                
                # modify 
                #sub_pd.at[index,3]=s[j+1].idx
                

In [ ]:
sub_18_abb=pd.DataFrame()
sub_18_abb=sub_18_abb.append(sub_pd)

In [ ]:
sub_18_abb=sub_18_abb.append(sub_pd)
sub_18_abb.shape


In [ ]:
sub_18_abb[0].unique()

In [ ]:
sub_18_abb.to_csv('sub_18_abb_modified.tsv',sep='\t')